# Creation d'un .json de l'archive GAMAR, avec virtualizarr / kerchunk

* Basé sur ls ~fpaul/tmp/create_virtualizarr_v2_riomar_icechunk

In [8]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [9]:
from functools import partial

import fsspec
import virtualizarr
import xarray as xr
import os

## Initialisation Dask

In [3]:
import distributed

cluster = distributed.LocalCluster()
client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 128,Total memory: 755.07 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33893,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:33089,Total threads: 8
Dashboard: http://127.0.0.1:36309/status,Memory: 47.19 GiB
Nanny: tcp://127.0.0.1:39569,


In [4]:
from virtualizarr.parsers import KerchunkJSONParser
from virtualizarr.registry import ObjectStoreRegistry
from obstore.store import LocalStore
from virtualizarr import open_virtual_dataset

import glob


#base_path = '/scale/project/lops-oh-fair2adapt/riomar/GAMAR/'
base_path = '/scale/project/lops-oh-fair2adapt/'

local_store = LocalStore()
registry = ObjectStoreRegistry({f"file://{base_path}": local_store})
#registry = ObjectStoreRegistry({})  # Pour test icechunk


#parser = KerchunkJSONParser()
#ou
from virtualizarr.parsers import HDFParser
parser = HDFParser()

In [17]:
base_path = '/scale/project/lops-oh-fair2adapt/riomar/GAMAR/'
#fpattern=base_path + 'GAMAR_1h_inst_Y2023M*.nc'
fpattern=base_path + 'GAMAR_1h_inst_Y20*M*.nc'
flist = glob.glob(fpattern)[:]
len(flist)

276

In [18]:
dask_open_virtual_dataset = partial(
    open_virtual_dataset,
    loadable_variables=["time_counter","time_instant","x_rho","y_rho","x_u","x_v","y_u","y_v","axis_nbounds"],
    parser=parser,
    registry=registry,
    decode_times=True,    
)
futures = client.map(dask_open_virtual_dataset, flist)
dss = client.gather(futures)
display(dss[:2])
display(dss[-2:])

[<xarray.Dataset> Size: 273GB
 Dimensions:              (axis_nbounds: 2, x_rho: 727, y_rho: 838, x_u: 726,
                           y_u: 838, x_v: 727, y_v: 837, time_counter: 696,
                           s_rho: 40, s_w: 41)
 Coordinates: (12/17)
   * axis_nbounds         (axis_nbounds) float32 8B 0.0 0.0
   * x_rho                (x_rho) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
   * y_rho                (y_rho) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
   * x_u                  (x_u) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
   * y_u                  (y_u) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
   * x_v                  (x_v) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
     ...                   ...
     nav_lat_rho          (y_rho, x_rho) float32 2MB ManifestArray<shape=(838,...
     nav_lon_rho          (y_rho, x_rho) float32 2MB ManifestArray<shape=(838,...
     nav_lat_u            (y_u, x_u) float32 2MB ManifestArray<shape=(838, 726...
     n

[<xarray.Dataset> Size: 263GB
 Dimensions:              (axis_nbounds: 2, x_rho: 727, y_rho: 838, x_u: 726,
                           y_u: 838, x_v: 727, y_v: 837, time_counter: 672,
                           s_rho: 40, s_w: 41)
 Coordinates: (12/17)
   * axis_nbounds         (axis_nbounds) float32 8B 0.0 0.0
   * x_rho                (x_rho) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
   * y_rho                (y_rho) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
   * x_u                  (x_u) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
   * y_u                  (y_u) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
   * x_v                  (x_v) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
     ...                   ...
     nav_lat_rho          (y_rho, x_rho) float32 2MB ManifestArray<shape=(838,...
     nav_lon_rho          (y_rho, x_rho) float32 2MB ManifestArray<shape=(838,...
     nav_lat_u            (y_u, x_u) float32 2MB ManifestArray<shape=(838, 726...
     n

In [19]:
%%skip version non dask
dss = [
    open_virtual_dataset(f,
        #concat_dim='time_counter',     # Dimension concaténation : time_counter
        #loadable_variables=['temp', 'nav_lat_rho', 'nav_lon_rho'],  # Seulement utiles
        loadable_variables=["time_counter","time_instant","x_rho","y_rho","x_u","x_v","y_u","y_v","axis_nbounds"],
        parser=parser,
        #combine='nested',           # compute nested = lent (hiérarchique), by_coords = rapide
        
        registry=registry,
                
        #coords="minimal",                # garde coords contenant time_counter seulement
        #data_vars="minimal",             # garde variables contenant time_counter seulement
        #compat="override",               # ignore différences entre fichiers (prend le 1er)
        #combine_attrs="drop_conflicts",  # supprime les attributs conflictuels
        decode_times=True,
    )
    for f in flist # ~5-10 secondes pour chaque fichier visiblement
]
dss

In [20]:
ds = (
    xr.concat(
        dss,
        dim="time_counter",
        compat="override",
        coords="minimal",
        combine_attrs="drop_conflicts",
    )
    .set_coords(["time_counter_bounds", "time_instant_bounds"])
)
ds

/tmp/ipykernel_1548044/212694250.py:2: FutureWarning: In a future version of xarray the default value for data_vars will change from data_vars='all' to data_vars=None. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set data_vars explicitly.
  xr.concat(


<xarray.Dataset> Size: 79TB
Dimensions:              (time_counter: 201600, axis_nbounds: 2, s_rho: 40,
                          s_w: 41, y_rho: 838, x_rho: 727, y_u: 838, x_u: 726,
                          y_v: 837, x_v: 727)
Coordinates: (12/19)
  * time_counter         (time_counter) datetime64[ns] 2MB 2020-02-01T00:52:3...
  * axis_nbounds         (axis_nbounds) float32 8B 0.0 0.0
    s_rho                (s_rho) float32 160B ManifestArray<shape=(40,), dtyp...
    s_w                  (s_w) float32 164B ManifestArray<shape=(41,), dtype=...
  * y_rho                (y_rho) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
  * x_rho                (x_rho) float32 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...                   ...
    nav_lat_rho          (y_rho, x_rho) float32 2MB ManifestArray<shape=(838,...
    nav_lon_rho          (y_rho, x_rho) float32 2MB ManifestArray<shape=(838,...
    nav_lat_u            (y_u, x_u) float32 2MB ManifestArray<shape=(838, 726...
    nav_lon_u            (y_u, x_u) float32 2MB ManifestArray<shape=(838, 726...
    nav_lat_v            (y_v, x_v) float32 2MB ManifestArray<shape=(837, 727...
    nav_lon_v            (y_v, x_v) float32 2MB ManifestArray<shape=(837, 727...
Data variables: (12/14)
    hc                   (time_counter) float32 806kB ManifestArray<shape=(20...
    theta_s              (time_counter) float32 806kB ManifestArray<shape=(20...
    theta_b              (time_counter) float32 806kB ManifestArray<shape=(20...
    Tcline               (time_counter) float32 806kB ManifestArray<shape=(20...
    Vtransform           (time_counter) float32 806kB ManifestArray<shape=(20...
    Cs_r                 (time_counter, s_rho) float32 32MB ManifestArray<sha...
    ...                   ...
    sc_w                 (time_counter, s_w) float32 33MB ManifestArray<shape...
    zeta                 (time_counter, y_rho, x_rho) float32 491GB ManifestA...
    u                    (time_counter, s_rho, y_u, x_u) float32 20TB Manifes...
    v                    (time_counter, s_rho, y_v, x_v) float32 20TB Manifes...
    temp                 (time_counter, s_rho, y_rho, x_rho) float32 20TB Man...
    salt                 (time_counter, s_rho, y_rho, x_rho) float32 20TB Man...
Attributes: (12/39)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

In [21]:
#outpath = '/home/datawork-lops-riomar/GAMAR'

outpath = '/scale/project/lops-oh-fair2adapt/fpaul/tmp/'
outfile = os.path.join(outpath,"riomar.json")
ds.virtualize.to_kerchunk(outfile, format="json")

## Quick check json catalog

In [22]:
#%env outfile={outfile}  
# Exporte en variable d'environnement pour le %%bash ensuite

In [23]:
import os
os.environ['SHELL'] = '/bin/bash'
#%env SHELL=/bin/bash

In [24]:
!ls -alh {outfile}

-rw-r--r-- 1 fpaul fair2adapt 359M Dec 11 11:28 /scale/project/lops-oh-fair2adapt/fpaul/tmp/riomar.json


In [28]:
%%bash
cd /scale/project/lops-oh-fair2adapt/fpaul/tmp
./make_sed_web.sh
ls -alh *.json

-rw-r--r-- 1 fpaul fair2adapt 359M Dec 11 11:28 riomar.json
-rw-r--r-- 1 fpaul fair2adapt 359M Dec 11 11:32 riomar_sed_web.json


In [25]:
import json
from itertools import islice

fichier = outfile

with open(fichier, "r", encoding="utf-8") as f:
    data = json.load(f)  # charge le JSON même sur une seule ligne
    texte = json.dumps(data, indent=2, ensure_ascii=False)

for ligne in islice(texte.splitlines(), 20):  # 40 premières lignes
    print(ligne)

{
  "version": 1,
  "refs": {
    ".zgroup": "{\"zarr_format\":2}",
    ".zattrs": "{\"name\":\"GAMAR_GLORYS_1h_inst\",\"description\":\"Created by xios\",\"Conventions\":\"CF-1.6\",\"title\":\"GAMAR_GLORYS\",\"rst_file\":\"croco_rst.nc\",\"grd_file\":\"croco_grd.nc\",\"ini_file\":\"croco_ini.nc\",\"frc_file\":\"croco_frc.nc\",\"VertCoordType\":\"NEW\",\"theta_s\":6.0,\"theta_s_expl\":\"S-coordinate surface control parameter\",\"theta_b\":4.0,\"theta_b_expl\":\"S-coordinate bottom control parameter\",\"Tcline\":15.0,\"Tcline_expl\":\"S-coordinate surface\\/bottom layer width\",\"Tcline_units\":\"meter\",\"hc\":15.0,\"hc_expl\":\"S-coordinate parameter, critical depth\",\"hc_units\":\"meter\",\"sc_w\":\"have a look at variable sc_w in this file\",\"Cs_w\":\"have a look at variable Cs_w in this file\",\"sc_r\":\"have a look at variable sc_r in this file\",\"Cs_r\":\"have a look at variable Cs_r in this file\",\"ndtfast\":60,\"tnu4_expl\":\"biharmonic mixing coefficient for tracers\",\"un

# FIN